<a href="https://colab.research.google.com/github/Theknowranking/WebCrawler/blob/main/CS429Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapy
# some functions were generated using ChatGPT (https://chat.openai.com/)
# extra explanations are provided for some functions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.8/287.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 11.5 MB/s eta 0:00:00


In [ ]:
!scrapy startproject webcrawler

New Scrapy project 'webcrawler', using template directory '/usr/local/lib/python3.10/dist-packages/scrapy/templates/project', created in:
    /content/webcrawler

You can start your first spider with:
    cd webcrawler
    scrapy genspider example example.com


In [ ]:
# execute this twice -> /content/webcrawler/webcrawler
%cd webcrawler

/content/webcrawler/webcrawler


In [ ]:
!scrapy crawl html_crawler -a seed_url=http://quotes.toscrape.com/ -a max_pages=100 -a max_depth=3

2024-04-22 06:09:41 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: webcrawler)
2024-04-22 06:09:41 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.1.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2024-04-22 06:09:41 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-22 06:09:41 [asyncio] DEBUG: Using selector: EpollSelector
2024-04-22 06:09:41 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2024-04-22 06:09:41 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2024-04-22 06:09:41 [scrapy.extensions.telnet] INFO: Telnet Password: 1deff268d55c1b8f
2024-04-22 06:09:41 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 '

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Optional: from gensim.models import Word2Vec
# Optional: import faiss

class Indexer:
    def __init__(self):
        self.documents = []
        self.tfidf_vectorizer = TfidfVectorizer()
        # Optional: Initialize Word2Vec model
        # Optional: Initialize FAISS index

    def add_document(self, document):
        self.documents.append(document)

    # Function to tokenize documents
    def preprocess_data(self):
        lemmatizer = WordNetLemmatizer()
        processed_documents = []
        for doc in self.documents:
            tokens = word_tokenize(doc.lower())
            tokens = [token for token in tokens if token.isalpha()]  # Remove punctuation
            tokens = [token for token in tokens if token not in stopwords.words('english')]  # Remove stopwords
            tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatization
            processed_doc = ' '.join(tokens)
            processed_documents.append(processed_doc)
        self.documents = processed_documents

    def build_index(self):
        # Preprocess data
        self.preprocess_data()

        # TF-IDF Vectorization
        tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.documents)

        # Optional: Train Word2Vec model
        # Optional: Train FAISS index

        # Save TF-IDF matrix and vectorizer to pickle
        with open('tfidf_matrix.pickle', 'wb') as f:
            pickle.dump(tfidf_matrix, f)

        with open('tfidf_vectorizer.pickle', 'wb') as f:
            pickle.dump(self.tfidf_vectorizer, f)

    def search(self, query):
        # Load TF-IDF matrix and vectorizer from pickle
        with open('tfidf_matrix.pickle', 'rb') as f:
            tfidf_matrix = pickle.load(f)

        with open('tfidf_vectorizer.pickle', 'rb') as f:
            tfidf_vectorizer = pickle.load(f)

        # Convert query to TF-IDF representation
        query_tfidf = tfidf_vectorizer.transform([query])

        # Calculate cosine similarity between query and documents
        similarity_scores = cosine_similarity(query_tfidf, tfidf_matrix)

        # Optional: Perform kNN similarity search using FAISS

        # Return document indices sorted by similarity score
        return similarity_scores.argsort()[0][::-1]


In [ ]:
# Example usage
indexer = Indexer()
indexer.add_document("This is the first document.")
indexer.add_document("This document is the second document.")
indexer.add_document("And this is the third one.")
indexer.build_index()

In [ ]:
# Search for a query
query = "second document"
results = indexer.search(query)
print("Search Results:")
for idx in results:
    print(indexer.documents[idx])

Search Results:
document second document
first document
third one


In [ ]:
from flask import Flask, request, jsonify
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)

# Sample documents
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one."
]

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Error messages
ERROR_INVALID_JSON = {"error": "Invalid JSON format. Please provide 'query' field."}
ERROR_NO_RESULTS = {"error": "No results found."}

def preprocess_text(text):
    # Tokenization, stopword removal, lemmatization, etc.
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha()]  # Remove punctuation
    tokens = [token for token in tokens if token not in stopwords.words('english')]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatization
    return ' '.join(tokens)

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def expand_query(query):
    expanded_query = []
    for word in query.split():
        expanded_query.append(word)
        synonyms = get_synonyms(word)
        expanded_query.extend(synonyms)
    return ' '.join(expanded_query)

@app.route('/process_query', methods=['POST'])
def process_query():
    # Validate JSON format
    if not request.is_json:
        return jsonify(ERROR_INVALID_JSON), 400

    data = request.get_json()
    if 'query' not in data:
        return jsonify(ERROR_INVALID_JSON), 400

    query = data['query']

    # Optional: Spelling correction
    # query = correct_spelling(query)

    # Optional: Query expansion
    # query = expand_query(query)

    # Preprocess query
    preprocessed_query = preprocess_text(query)

    # Convert query to TF-IDF representation
    query_tfidf = tfidf_vectorizer.transform([preprocessed_query])

    # Calculate cosine similarity between query and documents
    similarity_scores = cosine_similarity(query_tfidf, tfidf_matrix)
    ranked_indices = similarity_scores.argsort()[0][::-1]

    # Return top-K ranked results
    top_k = min(len(documents), 3)  # Return top 3 results
    results = []
    for idx in ranked_indices[:top_k]:
        results.append({"document": documents[idx], "similarity_score": similarity_scores[0][idx]})

    if not results:
        return jsonify(ERROR_NO_RESULTS), 404

    return jsonify(results), 200

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
